In [1]:
import numpy as np
from scipy.stats import norm
import scipy.integrate as integrate
from scipy import optimize
#from financepy.models.black import *
from financepy.utils.global_types import *

####################################################################
#  FINANCEPY BETA Version 0.370 - This build: 28 Oct 2024 at 20:26 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [2]:
def ivol_helper(K):
    return 0.510 - 0.591*K + 0.376*K**2 - 0.105*K**3 + 0.011*K**4

In [3]:
def ivol_HW6(K):
    if K>=3:
        return ivol_helper(3)
    else:
        return ivol_helper(K)

In [4]:
def Black(f:float, 
              k:float, 
              t:float,
              vol:float, 
              df:float,
              callorput):
    
    # Refer to last line in slide 16. This is NOT the same d1 used in BlackScholes
    d1 = (np.log(f/k) + 0.5 * (vol**2) * t) / (vol*np.sqrt(t))
    
    d2 = d1 - vol * np.sqrt(t)
    
    if callorput == "CALL":
        # Last line in slide 16
        value = df * (f * norm.cdf(d1) - k * norm.cdf(d2))
    else:
        # Last line in slide 16, but order is flipped, just like BlackScholes put
        value = df * (k * norm.cdf(-d2) - f * norm.cdf(-d1))
    
    return value

In [5]:
def black_with_smile(f, k, t, df, callorput):
    vol = ivol_HW6(k)
    return Black(f, k, t, vol, df, callorput) * df

In [6]:
# def EUROPEAN_CALL(S_0, K, r, sigma, T):
#     d1 = (np.log(S_0/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
#     d2 = d1 - sigma*np.sqrt(T)
    
#     return S_0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

# def EUROPEAN_PUT(S_0, K, r, sigma, T):
#     d1 = (np.log(S_0/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
#     d2 = d1 - sigma*np.sqrt(T)
    
#     return K*np.exp(-r*T)*norm.cdf(-d2) - S_0*norm.cdf(-d1)

In [7]:
def numerical_integration_HW6Q1(S0, r, q, T, SD):
    DF = np.exp(-r*T)
    DivF = np.exp(-q*T)
    f = S0*DivF/DF
    
    vol_for_range = ivol_HW6(f)
    maxS = f * np.exp(vol_for_range * SD * np.sqrt(T))
    forward_part = np.sqrt(f) * DF
    
    integrand_put = lambda y: y**(-1.5)/4 * black_with_smile(f, y, T, DF, "PUT")
    put_part, error = integrate.quad(integrand_put, 0.0001, f)
    
    integrand_call = lambda x: x**(-1.5)/4 * black_with_smile(f, x, T, DF, "CALL")
    call_part, error = integrate.quad(integrand_call, f, maxS)
    
    return forward_part - put_part - call_part

In [8]:
def numerical_integration_HW6Q2(S0, r, q, T, SD):
    DF = np.exp(-r*T)
    DivF = np.exp(-q*T)
    f = S0*DivF/DF
    
    vol_for_range = ivol_HW6(f)
    maxS = f * np.exp(vol_for_range * SD * np.sqrt(T))
    forward_part = f * f * f * DF
    
    integrand_put = lambda y: 6 * y * black_with_smile(f, y, T, DF, "PUT")
    put_part, error = integrate.quad(integrand_put, 0, f)
    
    integrand_call = lambda x: 6 * x * black_with_smile(f, x, T, DF, "CALL")
    call_part, error = integrate.quad(integrand_call, f, maxS)
    
    return forward_part + put_part + call_part

In [9]:
# driver routine for Q1 and Q2
q = 0.0; 
r = 0.0; 
T = 4; 
S0 = 1
DF = np.exp(-r*T)

SDs = np.linspace(1, 6, 6)

In [10]:
Q1numIntResults = [numerical_integration_HW6Q1(S0, r, q, T, sd) for sd in SDs]

In [11]:
Q1numIntResults

[0.9744529379057193,
 0.9737922020460309,
 0.9737631318280784,
 0.9737624998879959,
 0.9737624966184146,
 0.9737624965892451]

In [12]:
Q2numIntResults = [numerical_integration_HW6Q2(S0, r, q, T, sd) for sd in SDs]

In [13]:
Q2numIntResults

[1.4563473950073735,
 1.5157020030493702,
 1.5226686291504845,
 1.5230535930213192,
 1.5230590092868612,
 1.5230590334482683]